In [1]:
from src.dirs import MODELHOME, DATAHOME
from src.ml import to_dense, TargetEncoding, stemmer
from src.ml import evaluate_model, train_final_models
from src.io import load_artifacts

from src.data import create_features

from joblib import load
from sklearn.ensemble import StackingClassifier
from sklearn.svm import LinearSVC

from joblib import load
mdls = [d for d in MODELHOME.glob('*') if not d.is_file()]
mdls = {path.name: load_artifacts(path) for path in mdls}
try:
    del mdls['stack']
except:
    pass
print(mdls.keys())
binner = load(DATAHOME / 'processed' / 'binner.joblib')

dataset = load_artifacts(DATAHOME / 'final')
del dataset['path']
del dataset['name']
print(dataset.keys())

dict_keys(['nb-gau', 'voter', 'rf', 'gb', 'nb-multi', 'nb-ber'])
dict_keys(['x_ho', 'x_tr', 'y_ho', 'y_tr', 'x', 'y'])


In [11]:
for name, mdl in mdls.items():
    print(name) 
    print(mdl['holdout-score'])

nb-gau
{'score-tr': 0.6534312012208653, 'score-ho': 0.36152433196983863}
voter
{'score-tr': 0.6689681317498853, 'score-ho': 0.3991726793889182}
rf
{'score-tr': 0.9997975708502025, 'score-ho': 0.4039883339024996}
gb
{'score-tr': 0.6221273524145138, 'score-ho': 0.3825944808960168}
nb-multi
{'score-tr': 0.3516193098784601, 'score-ho': 0.3448433719911124}
nb-ber
{'score-tr': 0.3992375301523257, 'score-ho': 0.3730573641196602}


## Stacking

In [17]:
estimators = [
    (name, mdl['pipe-tr']) for name, mdl in mdls.items()
]

stack = StackingClassifier(
    estimators=estimators, 
    final_estimator=LinearSVC(C=2),
    stack_method='predict',
    verbose=2,
    n_jobs=6,
    cv=3
)

from sklearn.ensemble import VotingClassifier

stack.fit(dataset['x_tr'], dataset['y_tr'].values.reshape(-1, ))
evaluate_model(stack, **dataset)

/Users/adam/.pyenv/versions/3.7.6/envs/medium-headlines/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


{'score-tr': 0.4753715191282907, 'score-ho': 0.3898647653931139}

In [13]:
from src.ml import gridsearch, scorer

params = {'final_estimator__C': [0.5, 1]},
#gs = gridsearch(params, 'stack-search', pipe=stack, **dataset, verbose=2, n_jobs=6)

In [14]:
voter = VotingClassifier(
    estimators=estimators, 
    verbose=2,
    n_jobs=6,
)

voter.fit(dataset['x_tr'], dataset['y_tr'].values.reshape(-1, ))
evaluate_model(voter, **dataset)

{'score-tr': 0.6691645423232683, 'score-ho': 0.39299129427171725}

In [15]:
train_final_models(params=voter.get_params(), pipe=voter, **dataset, path='voter')

In [16]:
est = stack.final_estimator_

In [ ]:
#  5 classes
#  3 models
#  5 classes
est.coef_.shape
coef = est.coef_

In [ ]:
coef.reshape(5, -1, 5).sum(axis=2)